# Plot routes on SHN

See what highway corridors show up.

Can span multiple operators. These corridors, if improved, would benefit all operators.

In [ ]:
import branca
import geopandas as gpd
import intake
import pandas as pd

import create_calenviroscreen_lehd_data
from shared_utils import geography_utils 
from shared_utils import calitp_color_palette as cp

catalog = intake.open_catalog("*.yml")

# Modify rt_utils.ZERO_THIRTY_COLORSCALE to go up to 65 mph
ZERO_SIXTY_COLORSCALE = branca.colormap.step.RdYlGn_11.scale(vmin=0, vmax=65)
ZERO_SIXTY_COLORSCALE.caption = "Speed (miles per hour)"

# Map args
TILES = "CartoDB positron"

CUSTOM_CATEGORICAL = [
    cp.CALITP_CATEGORY_BRIGHT_COLORS[2], # yellow
    cp.CALITP_CATEGORY_BRIGHT_COLORS[1], # orange
    cp.CALITP_CATEGORY_BRIGHT_COLORS[0], # blue
    cp.CALITP_CATEGORY_BRIGHT_COLORS[5], # purple    
]

In [ ]:
gdf = catalog.highway_segment_stats.read()

gdf = gdf.assign(
    geometry = gdf.geometry.to_crs(
        geography_utils.CA_StatePlane).buffer(300).to_crs(geography_utils.WGS84)
)

In [ ]:
stats_cols = [
    "trips_peak", "trips_all_day", 
    "stop_arrivals_peak", "stop_arrivals_all_day",
    "stops_peak", "stops_all_day",
    "mean_speed_mph_trip_weighted",
]

def get_quartiles_by_district(gdf: gpd.GeoDataFrame, 
                              plot_col: list) -> gpd.GeoDataFrame:
    
    gdf_with_quartiles = gpd.GeoDataFrame()
    
    for i in sorted(gdf.District.unique()):
        district = gdf[gdf.District==i]
        
        quartiles = create_calenviroscreen_lehd_data.define_equity_groups(
            district, percentile_col = plot_col, num_groups = 4
        )
        
        gdf_with_quartiles = pd.concat(
            [gdf_with_quartiles, quartiles], 
            axis=0, ignore_index=True)
    
    return gdf_with_quartiles


gdf = get_quartiles_by_district(gdf, stats_cols)

In [ ]:
def plot_highway_corridor(gdf: gpd.GeoDataFrame, 
                          hwy_route: int, plot_col: str):
    """
    Returns a folium.Map using geopandas.explore()
    """
    plot_df = gdf[gdf.Route==hwy_route]
        
    if plot_col == "mean_speed_mph_trip_weighted": 
        m = plot_df.explore(
            plot_col, 
            # switch out colormap to allow higher speeds
            cmap = ZERO_SIXTY_COLORSCALE, 
            categorical=False, tiles = TILES)
    else:        
        m = plot_df.explore(
            f"{plot_col}_group", 
            cmap = CUSTOM_CATEGORICAL, 
            # quartiles....must be between 1-4
            vmin = 1, vmax = 4, 
            # set to True or else it's continuous, can't distinguish colors 
            categorical=True, tiles = TILES)
    
    display(m)

Highway 1 in District 4, near Golden Gate Park.
PCH (1) in District 7.
Both are technically part of SHN, but have much slower speeds, or are more like arterial segments. 

Change colormap to go above 30 mph because that was more applicable to urban speeds.

Maybe different scales by `RouteType`, because Interstate/US vs State is different speeds. See how speed limit can be brought in.

In [ ]:
for stat in stats_cols:
    if 'peak' in stat or 'speed' in stat:
        print(f"{stat.replace('_', ' ').title()}")
        plot_highway_corridor(gdf[gdf.District.isin([7, 12])], 
                              hwy_route=1, plot_col=stat)

## District Maps

Stick with either `trips_all_day` or `trips_peak` for now.

In [ ]:
def get_district_top_highways(district: int = 7, n: int = 5) -> tuple[int, list]:
    top_highways = gdf[(gdf.District==district)].Route.value_counts()[:n].index
    
    return district, top_highways

In [ ]:
def plot_for_district(district: int, n: int = 5, stat: list = ["trips_all_day"]): 
    district, hwys_list = get_district_top_highways(district=district, n=n)
    
    for i in range(0, len(hwys_list)):
        hwy_route = hwys_list[i]
        
        for s in stat:
            print(f"{s.replace('_', ' ').title()} on Hwy {hwy_route}")
            
            plot_highway_corridor(gdf[gdf.District == district], 
                                  hwy_route=hwy_route, plot_col=s)

In [ ]:
plot_for_district(district = 4, n = 5, 
                  stat=["trips_all_day", "mean_speed_mph_trip_weighted"])

In [ ]:
plot_for_district(district = 7, n = 5, stat=["trips_all_day", 
                                            "mean_speed_mph_trip_weighted"])